In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras import models
from keras import layers

2022-12-16 02:03:14.240396: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-16 02:03:15.661574: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/keesaev/miniconda3/lib/
2022-12-16 02:03:15.661628: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/keesaev/miniconda3/lib/
2022-12-16 02:03:15.661633: W tensorflow/compiler/tf2tensorrt/utils/

Загрузим датасет imdb и совместим данные для обучения с данными для валидации:

In [2]:
from keras.datasets import imdb
(training_data, training_targets), (testing_data,testing_targets) = imdb.load_data(num_words=10000, path="imdb1.npz")
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets),
axis=0)

Отобразим:
- Категории
- Число уникальных слов
- Среднюю длину отзыва
- Среднее отклонение длины отзыва

In [3]:
print("Categories:", np.unique(targets))
print("Number of unique words:",
len(np.unique(np.hstack(data))))
length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Categories: [0 1]
Number of unique words: 9998
Average Review length: 234.75892
Standard Deviation: 173


Отобразим первый отзыв:

In [4]:
print("Label:", targets[0])
print(data[0])

Label: 1
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


Декодируем слова с помощью словаря ```imdb.get_word_index```:

In [5]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()])
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[0]] )
print(decoded)

# this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert # is an amazing actor and now the same being director # father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for # and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also # to the two little boy's that played the # of norman and paul they were just brilliant children are often left out of the # list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

Векторизируем каждый обзор и заполним нулями недостающие элементы до 10000 и разделим данные на датасет обучения (10к отзывов) и для тестирования (10к отзывов):

In [6]:
num_of_words = 10000

def vectorize(sequences, dimension = num_of_words):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1
  return results
data_vectorized = vectorize(data)
targets = np.array(targets).astype("float32")

test_x = data_vectorized[:10000]
test_y = targets[:10000]
train_x = data_vectorized[10000:]
train_y = targets[10000:]

Обучим стандартную сеть и прогоним через неё тестовые данные:

In [7]:
from tensorflow.keras.models import Sequential

model1 = Sequential([
    layers.Dense(50, activation = "relu",input_shape=(num_of_words, )),
    layers.Dropout(0.3, noise_shape=None, seed=None),
    layers.Dense(50, activation = "relu"),
    layers.Dropout(0.2, noise_shape=None, seed=None),
    layers.Dense(50, activation = "relu"),
    layers.Dense(1, activation ="sigmoid")
])
model1.summary()

model1.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

history1 = model1.fit(
    train_x, train_y,
    epochs= 2,
    batch_size = 500,
    validation_data = (test_x, test_y)
)

print('[model1] val_acc: ', np.mean(history1.history['val_accuracy']), 
    'val_loss: ', np.mean(history1.history['val_loss']))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                500050    
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 505,201
Trainable params: 505,201
Non-trai

2022-12-16 02:03:22.174844: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2022-12-16 02:03:22.174889: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: keesaev-asus-ubuntu
2022-12-16 02:03:22.174895: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: keesaev-asus-ubuntu
2022-12-16 02:03:22.175029: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.108.3
2022-12-16 02:03:22.175047: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.161.3
2022-12-16 02:03:22.175051: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 470.161.3 does not match DSO version 510.108.3 -- cannot find working devices in this config

Epoch 1/2


2022-12-16 02:03:22.774319: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1600000000 exceeds 10% of free system memory.


80/80 [==============================] - 2s 18ms/step - loss: 0.4000 - accuracy: 0.8231 - val_loss: 0.2597 - val_accuracy: 0.8959
Epoch 2/2
80/80 [==============================] - 1s 11ms/step - loss: 0.2128 - accuracy: 0.9189 - val_loss: 0.2629 - val_accuracy: 0.8937
[model1] val_acc:  0.8948000073432922 val_loss:  0.26134058833122253


Теперь запросим датасет с числом уникальных слов уменьшенным в 2 раза (5000 слов):

In [8]:
num_of_words = 5000

from keras.datasets import imdb
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=num_of_words, path="imdb2.npz")
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets),axis=0)

data_vectorized = vectorize(data, dimension=num_of_words)
targets = np.array(targets).astype("float32")

test_x = data_vectorized[:10000]
test_y = targets[:10000]
train_x = data_vectorized[10000:]
train_y = targets[10000:]

Обучим ту же сеть с новым числом слов:

In [9]:
from tensorflow.keras.models import Sequential

model2 = Sequential([
    layers.Dense(50, activation = "relu",input_shape=(num_of_words, )),
    layers.Dropout(0.3, noise_shape=None, seed=None),
    layers.Dense(50, activation = "relu"),
    layers.Dropout(0.2, noise_shape=None, seed=None),
    layers.Dense(50, activation = "relu"),
    layers.Dense(1, activation ="sigmoid")
])

model2.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

history2 = model2.fit(
    train_x, train_y,
    epochs= 2,
    batch_size = 500,
    validation_data = (test_x, test_y)
)

print('[model2] val_acc: ', np.mean(history2.history['val_accuracy']), 
    'val_loss: ', np.mean(history2.history['val_loss']))

Epoch 1/2


2022-12-16 02:03:31.896738: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 800000000 exceeds 10% of free system memory.


80/80 [==============================] - 1s 11ms/step - loss: 0.4271 - accuracy: 0.8021 - val_loss: 0.2731 - val_accuracy: 0.8913
Epoch 2/2
80/80 [==============================] - 1s 7ms/step - loss: 0.2445 - accuracy: 0.9035 - val_loss: 0.2662 - val_accuracy: 0.8907
[model2] val_acc:  0.8910000026226044 val_loss:  0.2696753442287445


Можно заметить что точность слегка упала

Теперь запросим датасет с числом уникальных слов = 15000:

In [10]:
num_of_words = 15000

from keras.datasets import imdb
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=num_of_words, path="imdb3.npz")
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets),axis=0)

data_vectorized = vectorize(data, dimension=num_of_words)
targets = np.array(targets).astype("float32")

test_x = data_vectorized[:10000]
test_y = targets[:10000]
train_x = data_vectorized[10000:]
train_y = targets[10000:]

In [11]:
from tensorflow.keras.models import Sequential

model3 = Sequential([
    layers.Dense(50, activation = "relu",input_shape=(num_of_words, )),
    layers.Dropout(0.3, noise_shape=None, seed=None),
    layers.Dense(50, activation = "relu"),
    layers.Dropout(0.2, noise_shape=None, seed=None),
    layers.Dense(50, activation = "relu"),
    layers.Dense(1, activation ="sigmoid")
])

model3.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

history3 = model3.fit(
    train_x, train_y,
    epochs= 2,
    batch_size = 500,
    validation_data = (test_x, test_y)
)

print('[model3] val_acc: ', np.mean(history3.history['val_accuracy']), 
    'val_loss: ', np.mean(history3.history['val_loss']))

Epoch 1/2


2022-12-16 02:03:39.893526: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2400000000 exceeds 10% of free system memory.


80/80 [==============================] - 3s 25ms/step - loss: 0.3916 - accuracy: 0.8292 - val_loss: 0.2624 - val_accuracy: 0.8921
Epoch 2/2
80/80 [==============================] - 1s 15ms/step - loss: 0.1970 - accuracy: 0.9262 - val_loss: 0.2631 - val_accuracy: 0.8963
[model3] val_acc:  0.8941999971866608 val_loss:  0.262739360332489


Можно заметить, что точность стала слегка лучше

Добавим метод для преобразования пользовательского текста к данным совместимым с tf и предсказанию на трёх моделях:

In [38]:
from tensorflow import keras 

index = imdb.get_word_index()

def encode(words):
  arr = []
  for word in words:
    arr.append(index.get(word) + 3)
  res = keras.preprocessing.sequence.pad_sequences([arr], value=0, padding="post")
  return res

def predict_custom(sentence):
  word_list = sentence.split()
  enc = encode(word_list)
  vec = vectorize(enc, 10000)
  print(sentence)
  print('[model1]: ',model1.predict(vec))
  vec = vectorize(enc, 5000)
  print('[model2]: ',model2.predict(vec))
  vec = vectorize(enc, 15000)
  print('[model3]: ',model3.predict(vec))

Оценим предсказания относительно двух групп текста: содержащих позитивные слова и негативные:

In [40]:
predict_custom("brilliant good best awesome cool")
predict_custom("bad awful worst")

brilliant good best awesome cool
1/1 [==============================] - 0s 22ms/step
[model1]:  [[0.91431236]]
1/1 [==============================] - 0s 18ms/step
[model2]:  [[0.9200928]]
1/1 [==============================] - 0s 16ms/step
[model3]:  [[0.88805294]]
bad awful worst
1/1 [==============================] - 0s 20ms/step
[model1]:  [[0.05746444]]
1/1 [==============================] - 0s 17ms/step
[model2]:  [[0.03391721]]
1/1 [==============================] - 0s 18ms/step
[model3]:  [[0.06741618]]


Как видно, первый датасет был классифицирован как позитивный, а второй - как негативный